# Bruk av geopandas med data fra NVDB og SSB

Demo av hvordan geopandas kan brukes med data fra [Nasjonal vegdatabank](https://www.vegvesen.no/fag/teknologi/Nasjonal+vegdatabank) (NVDB) og [Statistisk Sentralbyrå](http://www.ssb.no/natur-og-miljo/geodata) (SSB). 

Vi ønsker å finne hvilke turistveger som går gjennom hvilke tettsteder - og hvilke trafikkulykker som er registrert der. 

Gangen er som følger: 
1. Last ned data over tettsteder fra [SSB](http://www.ssb.no/natur-og-miljo/geodata
1. Last ned turistveger fra [NVDB api](https://www.vegvesen.no/nvdb/apidokumentasjon/)
1. Finn snittet av turistveger og tettsteder (hvilke turistveger som går gjennom hvilke tettsteder)
1. Finn trafikkulykkene på turistvegene som går gjennom tettstedene. 
    1. Finn BoundingBox for hvert av tettstedene som har en turistveg gjennom seg.  
    1. Hent trafikkulykker innenfor BBox. 
    1. Finn trafikkulykkene som ligger på turistvegens senterlinje
        1. Alternativt finn trafikkulykkene innenfor radius fra turistvegen. 
1. Lag fine plott og sånn


# Kom i gang  

## Linux med anaconda


Last ned siste [Anaconda versjon](https://www.anaconda.com/download/#linux) for python 3.6 eller høyere, installer med ```bash Anaconda*.sh```- [Mer detaljert oppskrift](https://www.digitalocean.com/community/tutorials/how-to-install-the-anaconda-python-distribution-on-ubuntu-16-04). 

* Geopandas: ```conda install -c conda-forge geopandas``` [Dokumentasjon](http://geopandas.org/install.html#installing-geopandas)
* **todo: Hent geojson-bibliotek

## Linux uten anaconda

Antagelsen her er at du har python3 installert, men at din standard pythoninstallasjon er python 2.7.

* [Geopandas](http://geopandas.org/) ```pip3 install geopandas```
* [Geojson](https://pypi.python.org/pypi/geojson) ```pip3 install geojson```



## Windows 

Last ned og installer siste [Anaconda versjon](https://www.anaconda.com/download) for python 3.6 eller høyere. Kjør installasjonen. 

**Todo: Skriv om bruk av windows-oppfinnelsen _anaconda prompt_ **

# Last ned data fra SSB

Dette steget kan også gjøres manuelt. Direkte lenke for nedlasting av 2016-datasett er http://www.ssb.no/natur-og-miljo/_attachment/286736?_ts=158d3354ee0 

*NB! Det vil ikke overraske meg om lenkeråte gjør at du uansett må inn på SSB og finne nedlastingslenker* 

### Hent SSB-data manuelt 

1. Gå inn på http://www.ssb.no/natur-og-miljo/geodata#Nedlasting_av_datasett_med_dynamiske_avgrensinger . 
1. Klikk på *Nedlasting av kart over tettsteder*. 
1. Klikk på det året du vil laste ned for. 
    1. Nedlasting skal starte automagisk... hvis ikke kan du prøve en annen nettleser, evt kopiere lenken og bruke verktøy som curl eller wget. 
1. Pakk ut zip-arkivet og legg mappen ```Tettsted2016``` sammen med øvrige filer (rotnivå i repos'et). Dvs at stien til shapefilen skal være ```Tettsted2016/Tettsted2016.shp```







In [1]:
## Last ned SSB-data
# Se https://stackoverflow.com/a/14260592 , det om python 3+
import requests
import zipfile
import io
import os

url = 'http://www.ssb.no/natur-og-miljo/_attachment/286736?_ts=158d3354ee0'

# Laster kun en gang: 
if not os.path.exists('Tettsted2016/Tettsted2016.shp'):
    r = requests.get(url)
    if r.ok: 
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall()
    else: 
        print( "Nedlasting feiler:", r.status_code, r.reason )
        

# Importer de bibliotekene som trengs


In [2]:
import geopandas as gpd
import pandas as pd
import nvdbapi      # Fra https://github.com/LtGlahn/nvdbapi-V2
import nvdb2geojson # Fra https://github.com/LtGlahn/nvdbapi-V2

# Les data

In [3]:
tettsted = gpd.read_file( 'Tettsted2016/Tettsted2016.shp')
turistveger = nvdbapi.nvdbFagdata(777)
turistveger_geojson = nvdb2geojson.fagdata2geojson( turistveger)

In [4]:
turist2 = gpd.GeoDataFrame.from_features( turistveger_geojson['features'])

# Finn turistveger i tettsteder 

http://geopandas.org/mergingdata.html , spatial joins. 

sjoin

In [16]:
tettsted.crs = {'init': 'epsg:25833'}
turist2.crs = {'init': 'epsg:25833'}

turist_i_tettsted = gpd.sjoin( turist2, tettsted, how='inner', op='intersects')
tettsted_m_turist = gpd.sjoin( tettsted, turist2, how='inner', op='intersects')


In [20]:
print( "Antall turistveger", len( turist2))
print( "Antall tettsteder", len(tettsted))
print( "Tettsteder med turistveger", len(tettsted_m_turist))
print( "Turistveger som går gjennom tettsteder", len(turist_i_tettsted))

Antall turistveger 527
Antall tettsteder 990
Tettsteder med turistveger 120
Turistveger som går gjennom tettsteder 120


In [22]:
tettsted_m_turist.head(3)


,Tot_Bef,TETTNR,Tettstedsn,Areal_km2,Beff_tett,geometry,index_right,Link turistveg,Navn,Status,...,kortform,metadata,nummer,region,status,stedfesting,strekningslengde,til_meter,vegavdeling,vegsegment nr
76,261,4578,Nærland,0.257395,1014.007332,(POLYGON ((-46009.19449999917 6542647.40069999...,92,http://www.nasjonaleturistveger.no/no/jaeren,Jæren,Nasjonal turistveg,...,1100 Fv507 hp1 m14891-17983,"{'sist_modifisert': '2017-09-27 10:05:49', 've...",507,3,V,"{'til_posisjon': 1.0, 'fra_posisjon': 0.614770...",3092,17983,11,2
138,7003,4631,Jørpeland,4.135783,1693.270709,(POLYGON ((-14439.73290000072 6578247.41059999...,160,http://www.nasjonaleturistveger.no/no/ryfylke,Ryfylke,Nasjonal turistveg,...,1100 Rv13 hp6 m10097-15567,"{'sist_modifisert': '2017-10-04 14:25:16', 've...",13,3,V,"{'til_posisjon': 0.576433033971742, 'fra_posis...",5470,15567,11,61
138,7003,4631,Jørpeland,4.135783,1693.270709,(POLYGON ((-14439.73290000072 6578247.41059999...,161,http://www.nasjonaleturistveger.no/no/ryfylke,Ryfylke,Nasjonal turistveg,...,1100 Rv13 hp6 m15883-15893,"{'sist_modifisert': '2017-10-04 14:25:16', 've...",13,3,V,"{'til_posisjon': 1.0, 'fra_posisjon': 0.885852...",10,15893,11,62


In [23]:
turist_i_tettsted.head(3)

,Link turistveg,Navn,Status,antall vegsegmenter,fra_meter,fylke,geometry,hp,id,kategori,...,strekningslengde,til_meter,vegavdeling,vegsegment nr,index_right,Tot_Bef,TETTNR,Tettstedsn,Areal_km2,Beff_tett
11,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,54,18,"LINESTRING Z (481851.5 7569211.71002 3.52, 481...",25,231728512,E,...,4647,4701,21,12,865,1794,7812,Kabelvåg,1.113858,1610.617537
22,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,0,18,LINESTRING Z (478559.76025 7566856.00293 7.929...,26,231728512,E,...,5083,5083,21,23,865,1794,7812,Kabelvåg,1.113858,1610.617537
11,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,54,18,"LINESTRING Z (481851.5 7569211.71002 3.52, 481...",25,231728512,E,...,4647,4701,21,12,866,4598,7814,Svolvær,2.377111,1934.280603


In [19]:
for index, row in turist_i_tettsted:
    stedfest = row['']

,Link turistveg,Navn,Status,antall vegsegmenter,fra_meter,fylke,geometry,hp,id,kategori,...,kortform,metadata,nummer,region,status,stedfesting,strekningslengde,til_meter,vegavdeling,vegsegment nr
index_right,,,,,,,,,,,,,,,,,,,,,
0,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,1545,18,"LINESTRING Z (493412.35986 7590141.93018 8.49,...",23,231728512,E,...,1800 Ev10 hp23 m1545-9795,"{'sist_modifisert': '2017-10-03 09:28:49', 've...",10,5,V,"{'til_posisjon': 1.0, 'fra_posisjon': 0.146718...",8250,9795,21,1
1,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,9258,18,"LINESTRING Z (458191.12012 7572273 28.79, 4581...",28,231728512,E,...,1800 Ev10 hp28 m9258-18950,"{'sist_modifisert': '2017-10-03 09:28:49', 've...",10,5,V,"{'til_posisjon': 1.0, 'fra_posisjon': 0.0, 're...",9692,18950,21,2
2,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,18950,18,LINESTRING Z (451751.39844 7574125.84424 16.75...,28,231728512,E,...,1800 Ev10 hp28 m18950-27818,"{'sist_modifisert': '2017-10-03 09:28:49', 've...",10,5,V,"{'til_posisjon': 1.0, 'fra_posisjon': 0.0, 're...",8868,27818,21,3
3,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,27818,18,LINESTRING Z (447154.62109 7567799.19141 27.10...,28,231728512,E,...,1800 Ev10 hp28 m27818-33744,"{'sist_modifisert': '2017-10-03 09:28:49', 've...",10,5,V,"{'til_posisjon': 0.614169256596438, 'fra_posis...",5926,33744,21,4
4,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,0,18,"LINESTRING Z (443042.0857 7563748.29844 17.97,...",1,231728512,F,...,1800 Fv825 hp1 m0-6262,"{'sist_modifisert': '2017-10-03 09:28:49', 've...",825,5,V,"{'til_posisjon': 1.0, 'fra_posisjon': 0.0, 're...",6262,6262,21,5
5,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,5689,18,"LINESTRING Z (438848.06787 7561054.77063 7.93,...",29,231728512,E,...,1800 Ev10 hp29 m5689-8954,"{'sist_modifisert': '2017-10-03 09:28:49', 've...",10,5,V,"{'til_posisjon': 0.849602321108262, 'fra_posis...",3265,8954,21,6
6,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,8954,18,"LINESTRING Z (436225.7 7560061.19 -60.7, 43616...",29,231728512,E,...,1800 Ev10 hp29 m8954-10439,"{'sist_modifisert': '2017-10-03 09:28:49', 've...",10,5,V,"{'til_posisjon': 1.0, 'fra_posisjon': 0.849602...",1485,10439,21,7
7,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,10439,18,"LINESTRING Z (435610.48145 7558833.14551 5.82,...",29,231728512,E,...,1800 Ev10 hp29 m10439-11203,"{'sist_modifisert': '2017-10-03 09:28:49', 've...",10,5,V,"{'til_posisjon': 0.0806237384691413, 'fra_posi...",764,11203,21,8
8,http://www.nasjonaleturistveger.no/no/lofoten,Lofoten,Nasjonal turistveg,40,0,18,LINESTRING Z (434925.25968 7558541.65399 9.540...,30,231728512,E,...,1800 Ev10 hp30 m0-8713,"{'sist_modifisert': '2017-10-03 09:28:49', 've...",10,5,V,"{'til_posisjon': 1.0, 'fra_posisjon': 0.080623...",8713,8713,21,9
